In [1]:
# Do our imports for the code
import sqlite3
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
#connect to UMT GBQ instance. 
service_path = 'C:\\Users\\markmcdiffett\\Desktop\\2021fallsemester\\Applieddataanalytics\\'
service_file = 'umt-msba-037daf11ee16.json' # change this to your authentication information  
#gbq_proj_id = 'wedgeproject-327716' # change this to your poroject. 
#gbq_dataset_id = 'wedge_users'
gbq_proj_id = 'umt-msba' # change this to your poroject. 
gbq_dataset_id = 'transactions'

# And this should stay the same. 
private_key =service_path + service_file



In [3]:
# Get your credentials
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# And create a client to talk to GBQ
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [ ]:
#################################### TABLE 1 ############################################
#create connection and create table
connection = sqlite3.connect('Wedge.db')
cursor = connection.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS salesbyhour
              (Date DATE, Hour INT, Total_spend FLOAT, Transactions INT, Items INT)''')
connection.commit()

In [ ]:
#delete current rows in table
cursor.execute("DELETE FROM salesbyhour")
connection.commit()

In [ ]:
#################################### TABLE 1 ############################################
# 1.	Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day,
#determine the total spend in the store, the number of transactions, and a count of the number of items.

#query and execute query
qry = """
SELECT EXTRACT(date from datetime) as date, EXTRACT(hour from datetime) as hour,
       ROUND(sum(total),2) as Total_spend, count(*) as transactions, ROUND(sum(quantity),0) as items 
FROM `umt-msba.wedge_transactions.transArchive_*` 
WHERE unitPrice > 0
GROUP BY date, hour
ORDER by date, hour

"""

# And we execute queries with `client.query`
query_job = client.query(
    qry,
    location="US",
)

In [ ]:

#################################### TABLE 1 ############################################ 
#Insert query results into database
for row in query_job:
    Date = row[0]
    Hour = row[1]
    Total_spend = row[2]
    Transactions = row[3]
    Items = row[4]
    cursor.execute(f"INSERT INTO salesbyhour VALUES('{Date}','{Hour}','{Total_spend}','{Transactions}','{Items}')")
    connection.commit()

In [ ]:
#################################### TABLE 2 ############################################ 
#create connection and create table
connection = sqlite3.connect('Wedge.db')
cursor = connection.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS salesbyowner
              (Card_no INT, Year INT, Month INT, Sales FLOAT, Transactions INT, Items INT)''')
connection.commit()

In [ ]:
#delete current rows in table
cursor.execute("DELETE FROM salesbyowner")
connection.commit()

In [ ]:
#################################### TABLE 2 ############################################ 
# 2.	Sales by owner by year by month: 
#A file that has the following columns: card_no, year, month, sales, transactions, and items.


qry = """
SELECT
  card_no,
  EXTRACT(year
  FROM
    datetime) AS Year,
  EXTRACT(month
  FROM
    datetime) AS Month,
  COUNT (*) AS Transactions,
  ROUND(SUM(total),2) AS Sales,
  ROUND(SUM(QUANTITY),0) AS Items,
FROM
  `umt-msba.transactions.transArchive*`
WHERE
  unitPrice > 0
  AND card_no != 3
  AND TRANS_STATUS != "V"
  AND TRANS_STATUS != "C"
  AND TRANS_STATUS != "R"
  AND TRANS_STATUS != "M"
  AND TRANS_STATUS !="J"
GROUP BY
  CARD_NO,
  YEAR,
  MONTH
ORDER BY
  CARD_NO,
  YEAR,
  MONTH

"""

# And we execute queries with `client.query`
query_job = client.query(
    qry,
    location="US",
)

In [ ]:
#################################### TABLE 2 ############################################ 
#Insert query results into database
for row in query_job:
    card_no = row[0]
    year = row[1]
    month = row[2]
    transactions = row[3]
    sales = row[4]
    items = row[5]
    cursor.execute(f"INSERT INTO salesbyowner VALUES('{card_no}','{year}','{month}','{sales}','{transactions}','{items}')")
connection.commit()

In [4]:
#################################### TABLE 3 ############################################
#create connection and create table
connection = sqlite3.connect('Wedge.db')
cursor = connection.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS productsales
              (UPC INT, Description TEXT, Department_Number INT, Department_Name TEXT, Year INT, Month INT, Sales INT, Items INT, Transactions INT)''')
connection.commit()


In [5]:
#delete current rows in table
cursor.execute("DELETE FROM productsales")
connection.commit()


In [6]:
#################################### TABLE 3 ############################################
#3.	Sales by product description by year by month: A file that has the following columns:
# upc, description, department number, department name, year, month, sales, transactions, and items.

qry = """
 SELECT
  TRIM(description),
  EXTRACT(YEAR
  FROM
    DATETIME) AS YEAR,
  EXTRACT(MONTH
  FROM
    DATETIME) AS MONTH,
  DEPARTMENT AS DeptNo,
  UPC,
  ROUND(SUM(TOTAL),2) AS SPENDING,
  SUM(QUANTITY) AS ITEMS,
  COUNT(*) AS TRANSACTIONS,
FROM
  `umt-msba.transactions.transArchive*` 
WHERE
  TRANS_STATUS != "V"
  AND TRANS_STATUS != "C"
  AND TRANS_STATUS != "R"
  AND TRANS_STATUS != "M"
  AND TRANS_STATUS !="J"
GROUP BY
  description,
  year,
  month,
  DEPARTMENT,
  upc
ORDER BY
  TRIM(DESCRIPTION),
  YEAR,
  MONTH


"""

# And we execute queries with `client.query`
query_job = client.query(
    qry,
    location="US",
)


    
            
    


In [7]:
#create dictionary to link department number with department {dep_num: dep_name}
department_name_dict = {}
with open("C:\\Users\\markmcdiffett\\Desktop\\2021fallsemester\\Applieddataanalytics\\Python and GBQ\\exercise-python-gbq-markmcdiffett\\department_lookup.csv","r") as a:
    for row in a:
        row = row.split(',')
        dep_num = row[0]
        dep_name = row[1].replace("\n","")
        department_name_dict[dep_num] = dep_name 
# create "no department" label for rows with department number 0
department_name_dict["0"] = "No Department"        
        

In [8]:
#################################### TABLE 3 ############################################
#CARD_NO ORDERED BY NUMBER OF ENTRIES
for row in query_job:
    description = row[0]
    year = row[1]
    month = row[2]
    department_number = str(int(row[3]))
    department_name = department_name_dict[department_number]
    upc = row[4]
    total_spend = row[5]
    items = row[6]
    transaction = row[7]
    cursor.execute(f"INSERT INTO productsales VALUES('{upc}','{description}','{department_number}','{department_name}','{year}','{month}','{total_spend}','{items}','{transaction}')")
connection.commit()